In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

2024-05-04 02:47:22.630524: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 02:47:22.630608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 02:47:22.759174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data = pd.read_csv('../input/final-sarcasm-dataset/Dataset.csv')
print(data.columns)
data = data[['Comments','Sarcasm']]
data.head()

Index(['Comments', 'Emojis', 'Text', 'Sarcasm'], dtype='object')


,Comments,Sarcasm
0,That's how we roll😎 . . . . . . . . #cinemadd...,1
1,😂😂😂 #Sarcasm,1
2,At my bro Barbara school . I'm only one can i...,1
3,And I don’t know why people get hurt 😄 #sarca...,1
4,Yay. I see a #cold in my near #future 😂 #joke...,1


In [3]:
data['Comments'] = data['Comments'].apply(lambda x: x.lower())

# Function to add space after every alphanumeric sequence
def add_space_after_alphanumeric(text):
    return re.sub(r'(\w+)', r'\1 ', text)

# Apply the function to the 'Comments' column
data['Comments'] = data['Comments'].apply(add_space_after_alphanumeric)

# data['Emojis'] = data['Emojis'].astype(str)

print(data[ data['Sarcasm'] == 1].size)
print(data[ data['Sarcasm'] == 0].size)

data.head()

18520
26140


,Comments,Sarcasm
0,that 's how we roll 😎 . . . . . . . . #cin...,1
1,😂😂😂 #sarcasm,1
2,at my bro barbara school . i 'm only o...,1
3,and i don ’t know why people get hurt ...,1
4,yay . i see a #cold in my near #future...,1


In [4]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' .,?/-!#&')
tokenizer.fit_on_texts(data['Comments'].values)
X = tokenizer.texts_to_sequences(data['Comments'].values)
X = pad_sequences(X)

In [5]:
embed_dim = 128
rnn_out = 512

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(SimpleRNN(rnn_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.build((None, X.shape[1]))
print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 37, 128)        │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 37, 128)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 512)            │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 585,218 (2.23 MB)

 Trainable params: 585,218 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
Y = pd.get_dummies(data['Sarcasm']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14961, 37) (14961, 2)
(7369, 37) (7369, 2)


In [7]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20


I0000 00:00:1714790893.450018     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


468/468 - 16s - 33ms/step - accuracy: 0.7854 - loss: 0.4880
Epoch 2/20


W0000 00:00:1714790901.600482     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


468/468 - 3s - 7ms/step - accuracy: 0.8402 - loss: 0.4123
Epoch 3/20
468/468 - 3s - 6ms/step - accuracy: 0.8475 - loss: 0.3975
Epoch 4/20
468/468 - 3s - 6ms/step - accuracy: 0.8496 - loss: 0.3895
Epoch 5/20
468/468 - 3s - 6ms/step - accuracy: 0.8497 - loss: 0.3859
Epoch 6/20
468/468 - 3s - 6ms/step - accuracy: 0.8501 - loss: 0.3903
Epoch 7/20
468/468 - 3s - 6ms/step - accuracy: 0.8509 - loss: 0.3835
Epoch 8/20
468/468 - 3s - 6ms/step - accuracy: 0.8513 - loss: 0.3860
Epoch 9/20
468/468 - 3s - 6ms/step - accuracy: 0.8513 - loss: 0.3858
Epoch 10/20
468/468 - 3s - 6ms/step - accuracy: 0.8515 - loss: 0.3825
Epoch 11/20
468/468 - 3s - 6ms/step - accuracy: 0.8518 - loss: 0.3824
Epoch 12/20
468/468 - 3s - 7ms/step - accuracy: 0.8515 - loss: 0.3830
Epoch 13/20
468/468 - 3s - 6ms/step - accuracy: 0.8511 - loss: 0.3828
Epoch 14/20
468/468 - 3s - 7ms/step - accuracy: 0.8507 - loss: 0.3863
Epoch 15/20
468/468 - 3s - 6ms/step - accuracy: 0.8519 - loss: 0.3807
Epoch 16/20
468/468 - 3s - 6ms/step - a

In [8]:
model.save("Models/RNN_text_and_emoji.h5")

In [9]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

184/184 - 2s - 10ms/step - accuracy: 0.8518 - loss: 0.3868
score: 0.39
acc: 0.85


In [10]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose=0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 66.996699669967 %
neg_acc 99.55257270693512 %


In [11]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 1s - 506ms/step
negative
